In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import os
import random

packs = [
    "/kaggle/input/ocean-waste-detection-1/pack1/pack1",
    "/kaggle/input/ocean-waste-detection-1/pack2/pack2",
    "/kaggle/input/ocean-waste-detection-1/pack3/pack3",
    "/kaggle/input/ocean-waste-detection-1/pack4/pack4",
    "/kaggle/input/ocean-waste-detection-1/pack5/pack5",
    "/kaggle/input/ocean-waste-detection-1/pack6/pack6",
    "/kaggle/input/ocean-waste-detection-1/pack7/pack7",
    "/kaggle/input/ocean-waste-detection-1/pack8/pack8",
    "/kaggle/input/ocean-waste-detection-1/project-5-at-2024-11-30-01-02-75c210ac"
]

train_ratio = 0.8
image_ext = ('.JPG', '.jpeg', '.png')

all_data = []

for pack in packs:
    sides = ['leftside', 'rightside'] if 'leftside' in os.listdir(pack) else ['']
    for side in sides:
        image_path = os.path.join(pack, side, 'images') if side else os.path.join(pack, 'images')
        label_path = os.path.join(pack, side, 'labels') if side else os.path.join(pack, 'labels')

        if not os.path.exists(image_path) or not os.path.exists(label_path):
            continue

        for image_file in os.listdir(image_path):
            if image_file.endswith(image_ext):
                image_full_path = os.path.join(image_path, image_file)
                label_full_path = os.path.join(label_path, image_file.rsplit('.', 1)[0] + '.txt')  # Меняем расширение
                if os.path.exists(label_full_path):
                    all_data.append((image_full_path, label_full_path))

if len(all_data) == 0:
    raise ValueError("Не найдено ни одного изображения с меткой!")

random.shuffle(all_data)

split_idx = int(len(all_data) * train_ratio)
train_data = all_data[:split_idx]
val_data = all_data[split_idx:]

with open('/kaggle/working/train.txt', 'w') as train_file:
    train_file.writelines([f"{image}\n" for image, _ in train_data])

with open('/kaggle/working/val.txt', 'w') as val_file:
    val_file.writelines([f"{image}\n" for image, _ in val_data])

print("Файлы train.txt и val.txt успешно созданы!")

Файлы train.txt и val.txt успешно созданы!


In [5]:
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.4/898.4 kB 24.5 MB/s eta 0:00:0000:01


In [6]:
!pip install gdown
!gdown --id 1a0UcWVQVVN3NVqN2gBdEyVmPVs3CSSUH
!pip install albumentations==1.4
!pip uninstall -y wandb

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1a0UcWVQVVN3NVqN2gBdEyVmPVs3CSSUH
To: /kaggle/working/data.yaml
100%|█████████████████████████████████████████| 92.0/92.0 [00:00<00:00, 356kB/s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.17
    Uninstalling albumentations-1.4.17:
      Successfully uninstalled albumentations-1.4.17
Found existing installation: wandb 0.18.3
Uninstalling wandb-0.18.3:
  Successfully uninstalled wandb-0.18.3


In [ ]:
from ultralytics import YOLO

model = YOLO('/kaggle/input/yolo10m/pytorch/default/1/best.pt')  

model.train(
    data='/kaggle/working/data.yaml',  
    epochs=10,             
    batch=8,
    save_period=1,
    device=0,
    imgsz=1024,
    single_cls=True,
    project='ocean_waste_detection',  
    name='exp',
    plots=True
)

In [ ]:
model = YOLO('/kaggle/input/yolo11n/pytorch/default/1/best.pt')
metrics = model.val(data='/kaggle/working/data.yaml', imgsz=1024, plots=True)

In [ ]:
metrics.box.map 
metrics.box.map50 
metrics.box.map75  
metrics.box.maps

In [ ]:
metrics.box.map50 

In [ ]:
metrics.box.map75  

In [7]:
from ultralytics import YOLO

model = YOLO('/kaggle/input/yolo11n/pytorch/default/1/best.pt')

results = model.predict(
    source='/kaggle/working/val.txt',
    save=True,                
    conf=0.0001,
    stream=True 
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
df = pd.DataFrame([box.data[0] for result in results for box in result.boxes],
                  columns=['class', 'x_center', 'y_center', 'width', 'height', 'confidence', 'iou'])

correct_detections = df[df['iou'] > 0.5]


image 1/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015158.JPG: 768x1024 (no detections), 456.1ms
image 2/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015160.JPG: 768x1024 (no detections), 283.0ms
image 3/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015165.JPG: 768x1024 (no detections), 276.7ms
image 4/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015169.JPG: 768x1024 (no detections), 283.3ms
image 5/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015170.JPG: 768x1024 (no detections), 264.9ms
image 6/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015173.JPG: 768x1024 (no detections), 268.5ms
image 7/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015174.JPG: 768x1024 (no detections), 276.6ms
image 8/2798 /kaggle/input/ocean-waste-detection-1/pack1/pack1/leftside/images/G0015180.JPG: 768x1024 (no dete

ValueError: 7 columns passed, passed data had 6 columns

In [32]:
#from ultralytics import YOLO
#
#model = YOLO('/kaggle/input/yolo11n/pytorch/default/1/best.pt')
#
#results = model(["/kaggle/input/ocean-waste-detection-1/project-5-at-2024-11-30-01-02-75c210ac/project-5-at-2024-11-30-01-02-75c210ac/images/222e23be-_IMG6712.JPG"], conf=0.0001)  # return a list of Results objects
#
#for result in results:
#    boxes = result.boxes  # Boxes object for bounding box outputs
#    masks = result.masks  # Masks object for segmentation masks outputs
#    keypoints = result.keypoints  # Keypoints object for pose outputs
#    probs = result.probs  # Probs object for classification outputs
#    obb = result.obb  # Oriented boxes object for OBB outputs
#    result.show()  # display to screen
#    result.save(filename="result5.jpg") 

In [ ]:
for r in results:
    r.

In [31]:
#from PIL import Image
#img = Image.open('//kaggle/input/ocean-waste-detection-1/project-5-at-2024-11-30-01-02-75c210ac/project-5-at-2024-11-30-01-02-75c210ac/images/222e23be-_IMG6712.JPG')
#display(img)

In [30]:
img.save('example.JPG')

In [ ]:
img.size

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh {HOME}/weights

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

In [ ]:
%cd {HOME}

!yolo task=detect mode=train epochs=10 batch=16 plots=True \
model={HOME}/weights/yolov10n.pt \
data={HOME}/data.yaml

In [ ]:
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113